<a href="https://colab.research.google.com/github/filmroyal/filmroyal/blob/main/breakoutD_4H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener matplotlib openpyxl
!pip install ta

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)

def breakout_strategy(data, period=20):
    df = data.copy()
    df['High_Max'] = df['High'].rolling(window=period).max()
    df['Low_Min'] = df['Low'].rolling(window=period).min()
    df['Breakout_Up'] = df['Close'] > df['High_Max'].shift(1)
    df['Breakout_Down'] = df['Close'] < df['Low_Min'].shift(1)
    return df

tv = TvDatafeed()
Hisseler = get_all_symbols(market='turkey')
Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
Hisseler = sorted(Hisseler)

intervals = [Interval.in_4_hour, Interval.in_daily]

df_breakout_up = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Breakout Türü', 'Zaman Dilimi'])
df_breakout_down = pd.DataFrame(columns=['Hisse Adı', 'Son Fiyat', 'Breakout Türü', 'Zaman Dilimi'])

for interval in intervals:
    for i in range(0, len(Hisseler)):
        try:
            data = tv.get_hist(symbol=Hisseler[i], exchange='BIST', interval=interval, n_bars=500)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data = data.reset_index()
            breakout_data = breakout_strategy(data, period=20)
            breakout_data['datetime'] = pd.to_datetime(breakout_data['datetime'])
            breakout_data.set_index('datetime', inplace=True)
            last_signal = breakout_data.iloc[-1]
            Breakout_Up = last_signal['Breakout_Up']
            Breakout_Down = last_signal['Breakout_Down']
            Last_Price = last_signal['Close']
            if Breakout_Up:
                df_breakout_up.loc[len(df_breakout_up)] = [Hisseler[i], Last_Price, 'Breakout Yukarı', interval.name]
            elif Breakout_Down:
                df_breakout_down.loc[len(df_breakout_down)] = [Hisseler[i], Last_Price, 'Breakout Aşağı', interval.name]
        except Exception as e:
            print(f"Error processing {Hisseler[i]}: {e}")
        print(f"Processing {Hisseler[i]}: {'Breakout Yukarı' if Breakout_Up else 'Breakout Aşağı' if Breakout_Down else 'No Signal'}")

# Breakout Yukarı ve Aşağı sinyallerini tek bir tabloya ekleyelim
df_all_signals = pd.concat([df_breakout_up, df_breakout_down]).reset_index(drop=True)

# Ekranda tablo olarak gösterme
print("Breakout Yukarı Sinyalleri:")
print(df_breakout_up)
print("\nBreakout Aşağı Sinyalleri:")
print(df_breakout_down)

# Excel dosyasına kaydetme
excel_file = 'breakout_signals.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    df_breakout_up.to_excel(writer, sheet_name='Breakout Yukarı', index=False)
    df_breakout_down.to_excel(writer, sheet_name='Breakout Aşağı', index=False)
    df_all_signals.to_excel(writer, sheet_name='Tüm Sinyaller', index=False)

print(f"\nBreakout yukarı ve aşağı sinyalleri '{excel_file}' dosyasına kaydedildi.")
